In [52]:
import os
import cv2
import numpy as np
from skimage.util import random_noise

## Costanti

In [50]:
PATH_DATASET_LAVORATO = './Dati Sensori/Dataset Sensori Lavorato'
PATH_DATASET_NOISE = './Dati Sensori/Dataset Sensori Noise'
PATH_DATASET_ART_RIF = './Dati Sensori/Dataset Sensori Artefatti Riflessione'
PATH_DATASET_SPECKLE = './Dati Sensori/Dataset Sensori Speckle'

## Configurazione

In [62]:
VAR_RUMORE_GAUSSIANO = 0.1
SPECKLE_MEAN = 0
SPECKLE_STD = 0.1

## Rumore gaussiano

In [34]:
def applica_rumore_gaussiano(path_img_orig, path_img_dist, varianza):
    print(f"Applico il rumore gaussiano a {path_img_orig} ...")
    img = cv2.imread(path_img_orig)
    img_distorta = random_noise(img, mode='gaussian', var=varianza)
    img_distorta = (255 * img_distorta).astype('uint8')
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    cv2.imwrite(path_img_dist, img_distorta)

In [35]:
%%time
for root, dirs, files in os.walk(PATH_DATASET_LAVORATO):
    for file in files:
        if file.endswith(".jpg"):
            path_img_orig = os.path.join(root, file)
            path_img_dist = path_img_orig.replace(PATH_DATASET_LAVORATO, PATH_DATASET_NOISE)
            path_img_dist = path_img_dist.replace('.jpg', '_noise.jpg')            
            applica_rumore_gaussiano(path_img_orig, path_img_dist, VAR_RUMORE_GAUSSIANO)

Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_3.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_2.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_1.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_5.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_4.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_6.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_7.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_10.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_11.jpg ...
Applico il rumore gaussiano a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_13.jpg .

## Artefatti di riflessione
Vengono modificati i valori dei pixel per riflettere l'immagine rispetto all'asse di simmetria.
L'immagine riflessa viene sovrapposta all'immagine originale per ottenere un effetto di fusione tra le due immagini.  
vd. https://onlinelibrary.wiley.com/doi/abs/10.7863/jum.1986.5.4.227

In [48]:
def applica_artefatti_riflessione(path_img_orig, path_img_dist):
        print(f"Applico artefatti di riflessione a {path_img_orig} ...")
        img = cv2.imread(path_img_orig)
        img_riflessa = img.copy()
        h, w, _ = img.shape
        centro_x = int(w/2)
        for y in range(h):
            for x in range(centro_x, w):
                x_rifl = w - x - 1  # Posizione x del pixel riflettuto
                img_riflessa[y, x] = img[y, x_rifl]
        blended_img = cv2.addWeighted(img, 0.5, img_riflessa, 0.5, 0)
        os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
        cv2.imwrite(path_img_dist, blended_img)

In [ ]:
%%time
for root, dirs, files in os.walk(PATH_DATASET_LAVORATO):
    for file in files:
        if file.endswith(".jpg"):
            path_img_orig = os.path.join(root, file)
            path_img_dist = path_img_orig.replace(PATH_DATASET_LAVORATO, PATH_DATASET_ART_RIF)
            path_img_dist = path_img_dist.replace('.jpg', '_artrif.jpg')            
            applica_artefatti_riflessione(path_img_orig, path_img_dist)

Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_3.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_2.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_1.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_5.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_4.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_6.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_7.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_10.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_11.jpg ...
Applico artefatti di riflessione a ./Dati Sensori/Dat

## Speckle

In [56]:
def applica_speckle(path_img_orig, path_img_dist, speckle_mean, speckle_std):
    print(f"Applico lo speckle a {path_img_orig} ...")
    img = cv2.imread(image_path, 0)
    speckle_noise = np.random.normal(speckle_mean, speckle_std, img.shape)
    img_rumore = img + img * speckle_noise
    img_rumore = cv2.normalize(img_rumore, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    os.makedirs(os.path.dirname(path_img_dist), exist_ok=True)
    cv2.imwrite(path_img_dist, img_rumore)

In [63]:
%%time
for root, dirs, files in os.walk(PATH_DATASET_LAVORATO):
    for file in files:
        if file.endswith(".jpg"):
            path_img_orig = os.path.join(root, file)
            path_img_dist = path_img_orig.replace(PATH_DATASET_LAVORATO, PATH_DATASET_SPECKLE)
            path_img_dist = path_img_dist.replace('.jpg', '_speckle.jpg')            
            applica_speckle(path_img_orig, path_img_dist, SPECKLE_MEAN, SPECKLE_STD)


Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_3.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_2.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_1.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_5.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_4.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_6.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_7.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_10.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_11.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_13.jpg ...
Applico lo speckle a ./Dati Sensori/Dataset Sensori Lavorato/9_M/9_M_1/9_M_1_img_12.jpg